In [11]:
import datetime as dt
import json
import time

import mplfinance as mpf
import pandas as pd
import requests
import sqlalchemy as sa

import shared

In [7]:
url_clickhouse = "clickhouse+native://default@clickhouse-1:9000/default"
engine_clickhouse = sa.create_engine(url_clickhouse)

### trade_emas


In [ ]:
run_id = "520cec67-0632-4f3d-bf8e-d2b140f8946b"
debugs_candles_df = (
    pd.read_sql_query(
        """
        SELECT *
        FROM default.debugs
        WHERE run_id = %(run_id)s AND kind = 'candle';
        """,
        engine_clickhouse,
        params={"run_id": run_id},
    )
    .assign(
        c=lambda x: x.content.apply(json.loads),
        timestamp=lambda x: x.c.apply(
            lambda x: dt.datetime.fromtimestamp(x["open_time"] / 1000)
        ),
        open=lambda x: x.c.apply(lambda x: x["open"]),
        close=lambda x: x.c.apply(lambda x: x["close"]),
        low=lambda x: x.c.apply(lambda x: x["low"]),
        high=lambda x: x.c.apply(lambda x: x["high"]),
    )
    .set_index("timestamp")
    .sort_index()
)
backtest_df = pd.read_sql_query(
    """
    SELECT *
    FROM default.debugs
    WHERE run_id = %(run_id)s AND kind = 'backtest'
    """,
    engine_clickhouse,
    params={"run_id": run_id},
).assign(
    c=lambda x: x.content.apply(json.loads),
    open_price=lambda x: x.c.apply(lambda x: x["open_price"]),
    open_timestamp_millis=lambda x: x.c.apply(lambda x: x["open_timestamp_millis"]),
    close_timestamp_millis=lambda x: x.c.apply(lambda x: x["close_timestamp_millis"]),
    reason=lambda x: x.c.apply(lambda x: x["reason"]),
    bottom_threshold=lambda x: x.c.apply(lambda x: x.get("bottom_threshold", 0.0)),
    trailing_threshold=lambda x: x.c.apply(lambda x: x.get("trailing_threshold", 0.0)),
)
profit_df = pd.read_sql_query(
    """
    SELECT
        JSONExtractFloat(content, 'open_price') open_price,
        COALESCE(
            JSONExtract(content, 'bottom_threshold', 'Nullable(Float64)'),
            JSONExtract(content, 'trailing_threshold', 'Nullable(Float64)'),
            -1.0
        ) close_price,
        close_price - open_price profit
    FROM default.debugs
    WHERE run_id = %(run_id)s AND kind = 'backtest'
    """,
    engine_clickhouse,
    params={"run_id": run_id},
)
alines_list = [
    [
        (dt.datetime.fromtimestamp(row.open_timestamp_millis / 1000), row.open_price),
        (
            dt.datetime.fromtimestamp(row.close_timestamp_millis / 1000),
            row.bottom_threshold or row.trailing_threshold,
        ),
    ]
    for _, row in backtest_df.iterrows()
]
alines_colors = ["g" if x[1][1] - x[0][1] > 0 else "r" for x in alines_list]
fig, axs = mpf.plot(
    debugs_candles_df,
    warn_too_much_data=len(debugs_candles_df),
    type="candle",
    figsize=(140, 40),
    style=shared.s,
    ema=12,
    alines={"alines": alines_list, "colors": alines_colors, "linewidths": 5},
    returnfig=True,
)
xticks = pd.date_range(
    debugs_candles_df.index.min(), debugs_candles_df.index.max(), freq="1h"
)
xticks_locations = [debugs_candles_df.index.get_loc(tick) for tick in xticks]
xticks_labels = [tick.isoformat() for tick in xticks]
_ = axs[-2].xaxis.set_ticks(xticks_locations)
_ = axs[-2].set_xticklabels(xticks_labels, fontsize=5, rotation=90)
print(f"profit: {profit_df.profit.sum()}")

In [302]:
# debugs_candles_df.head(1)
# backtest_df.head(1)
# profit_df.head(1)

### clickhouse DDLs


In [ ]:
"""
rpk topic create trades

CREATE TABLE default.trades_binance_raw
(
  read_topic String,
  read_error String,
  read_raw_message String,
  open_time DateTime,
  open Float64,
  high Float64,
  low Float64,
  close Float64,
  volume Float64,
  kline_close_time DateTime,
  quote_asset_volume Float64,
  number_of_trades Int64,
  taker_buy_base_asset_volume Float64,
  taker_buy_quote_asset_volume Float64,
  t Int64,
  symbol String,
  date_iso Date
)
ENGINE = MergeTree
PARTITION BY toDate(open_time)
ORDER BY toDate(open_time);

CREATE MATERIALIZED VIEW default.trades_binance_mv
TO default.trades_binance_raw AS
SELECT
  _topic read_topic,
  _error read_error,
  _raw_message read_raw_message,
  fromUnixTimestamp64Milli(JSONExtractUInt(data, 'open_time')) open_time,
  JSONExtractFloat(data, 'open') open,
  JSONExtractFloat(data, 'high') high,
  JSONExtractFloat(data, 'low') low,
  JSONExtractFloat(data, 'close') close,
  JSONExtractFloat(data, 'volume') volume,
  fromUnixTimestamp64Milli(toInt64(JSONExtractFloat(data, 'kline_close_time'))) kline_close_time,
  JSONExtractFloat(data, 'quote_asset_volume') quote_asset_volume,
  JSONExtractUInt(data, 'number_of_trades') number_of_trades,
  JSONExtractFloat(data, 'taker_buy_base_asset_volume') taker_buy_base_asset_volume,
  JSONExtractFloat(data, 'taker_buy_quote_asset_volume') taker_buy_quote_asset_volume,
  JSONExtractUInt(data, 't') t,
  JSONExtractString(data, 'symbol') symbol,
  toDate(JSONExtractString(data, 'date_iso')) date_iso
FROM default.trades_binance_queue;


CREATE TABLE default.trades_raw
(
  read_topic String,
  read_error String,
  read_raw_message String,
  id UInt64,
  price Float64,
  qty Float64,
  base_qty Float64,
  time DateTime,
  is_buyer Bool,
  is_maker Bool,
  exchange String,
  symbol String,
  date_iso Date
)
ENGINE = ReplacingMergeTree
PRIMARY KEY (id)
PARTITION BY toDate(time);

CREATE TABLE default.trades_queue
(data String)
ENGINE = Kafka
SETTINGS kafka_broker_list = 'redpanda-1:9093',
         kafka_topic_list = 'trades',
         kafka_group_name = 'clickhouse-consumer',
         kafka_format = 'JSONAsString',
         kafka_thread_per_consumer = 0,
         kafka_num_consumers = 1,
         kafka_handle_error_mode = 'stream',
         kafka_max_block_size = 100000;

CREATE MATERIALIZED VIEW default.trades_mv
TO default.trades_raw AS
SELECT
  _topic read_topic,
  _error read_error,
  _raw_message read_raw_message,
  JSONExtractUInt(data, 'id') id,
  JSONExtractFloat(data, 'price') price,
  JSONExtractFloat(data, 'qty') qty,
  JSONExtractFloat(data, 'base_qty') base_qty,
  fromUnixTimestamp64Milli(JSONExtractUInt(data, 'time')) time,
  JSONExtractBool(data, 'is_buyer') is_buyer,
  JSONExtractBool(data, 'is_maker') is_maker,
  JSONExtractString(data, 'exchange') exchange,
  JSONExtractString(data, 'symbol') symbol,
  toDate(JSONExtractString(data, 'date_iso')) date_iso
FROM default.trades_queue;


CREATE TABLE default.debugs
(
run_id String,
kind String,
content String
)
ENGINE = Log;


CREATE TABLE default.trades_t_raw
(
  id UInt64,
  price Float64,
  qty Float64,
  base_qty Float64,
  time DateTime,
  is_buyer Bool,
  is_maker Bool,
  exchange String,
  symbol String,
  date_iso Date
)
ENGINE = ReplacingMergeTree
PRIMARY KEY (id)
PARTITION BY toDate(time);
"""

### bybit funding rate


In [13]:
end_time_secs = int(time.time())
start_time_secs = end_time_secs - 60 * 60 * 24 * 7
window_secs = 60 * 60
shift_secs = 0
eth_funding_rates_responses = []
while start_time_secs + shift_secs <= end_time_secs:
    # curl 'https://api-testnet.bybit.com/v5/market/funding/history?category=linear&symbol=ETHPERP&limit=1' | python3.10 -m json.tool
    url = (
        "https://api.bybit.com"
        "/v5/market/funding/history?category=linear&symbol=ETHPERP&limit=1"
        f"&startTime={(start_time_secs + shift_secs)*1000}"
        f"&endTime={(start_time_secs + shift_secs + window_secs) * 1000}"
    )
    res = requests.get(url)
    res.raise_for_status()
    eth_funding_rates_responses.append(res.json())
    shift_secs += window_secs

In [ ]:
l = [
    # (x["symbol"], int(x["fundingRateTimestamp"]), float(x["fundingRate"]))
    x
    for res in eth_funding_rates_responses
    for x in res["result"]["list"]
]
(
    pd.DataFrame(l)
    .sort_values(by=["fundingRateTimestamp"])
    .assign(
        event_timestamp=lambda x: x.fundingRateTimestamp.astype(int).apply(
            lambda x: dt.datetime.fromtimestamp(x / 1000)
        )
    )
)

### commands


In [ ]:
# docker run --rm -p 8888:8888 -v "$(pwd)/jupy/.var":/home/jovyan/.var jupyterhub/singleuser:4.0.0b2 start-notebook.sh --NotebookApp.password=\"sha1:f8409da91c32:bab6de2e611c36ac6b49d3f8d8229689cd9d5474\"
# pip install spylon-kernel pandas==2.2.0 matplotlib==3.8.2